In [45]:
import sympy as sp
from sympy import Matrix as spmtx
from sympy import symbols as syms
import math
from collections import deque
import copy
import random
import numpy as np

In [32]:
avaliable_classes = ['mono_poly']

In [162]:
def condition_num(A, norm = None):
    min_dim = min(A.shape)
    if A.rank() != min_dim:
        return -1 # since condition number is geq 1, this indicate it is not full rank
    else:
        cond = A.pinv().norm() * A.norm()
        return cond

In [33]:
def get_class_name(obj):
    return obj.__class__.__name__

In [170]:
class mono_poly:
    
    def _gen_terms(self, var_list, total_degree, term):
        '''
        var_list is of type deque
        '''
        terms = []
        if len(var_list) == 1:
            term *= var_list[0] ** total_degree
            terms.append(term)
            return terms
        elif total_degree == 0:
            term *= 1
            terms.append(term)
            return terms
        else:
            for i in range(total_degree, -1, -1):
                var_list_further = copy.deepcopy(var_list)
                var = var_list_further.popleft()
                term_further = term * var ** i
                terms += self._gen_terms(var_list_further, total_degree - i, term_further)
            return terms
    
    def __init__(self, deg, var_num, coeff=None):
        '''
        Init a multivariate polynomial with given num of var and num of degree.
        The base used here is monomial
        '''
        self.degree = deg
        self.var_num = var_num
        var_list = deque()
        for i in range(var_num):
            var_list.append(syms('x_' + str(i)))
        self.var_list = var_list
        terms = []
        for i in range(deg, -1, -1):
            terms += self._gen_terms(self.var_list, i, 1)
        self.base = terms
        self.term = terms
        self.base_vec = spmtx(terms)
        if coeff == None:
            pass
            # coeff = []
            # for i in range(len(terms)):
            #     coeff.append(random.randint(-100, 100))
            # coeff = numpy.array(coeff)
        self.coeff = coeff
    
    def c_extract_map(self, poly, basis):
        """
        Return the coeff of the poly input. 
        """
        c = poly.coeff(basis)
        if len(c.free_symbols) == 0:
            return c
        else:
            return sp.core.numbers.Zero()

In [147]:
def gen_mmoment_matrix(base_vec):
    return base_vec * spmtx.transpose(base_vec)

In [148]:
def a_vec(m_mtx, base_type, basis):
    a = []
    for i in range(0, m_mtx.shape[0]):
        for j in range(i, m_mtx.shape[1]):
            coeff = base_type.c_extract_map(m_mtx[i, j], basis)
            if i != j:
                coeff *= 2
            a.append(coeff)
    return a

In [160]:
def A_mtx(m_mtx, base_type, base):
    A = []
    for b in base:
        A.append(a_vec(m_mtx, base_type, b))
    return spmtx(A)

In [161]:
r = mono_poly(2, 2)

m_mtx = gen_mmoment_matrix(r.base_vec)

base = mono_poly(deg=4, var_num = 2)

base.base

b = base.base[0]
args = m_mtx[0].coeff(b).args

base.c_extract_map(m_mtx[0,0], b)

gather_a_vec(m_mtx, base_type=base, basis=base.base[0])

A = A_mtx(m_mtx, base_type=base, base=base.base)

In [164]:
condition_num(A).evalf()

19.0092082949291

In [165]:
mono_poly

__main__.mono_poly

In [186]:
def experiment(m_mtx_base, poly_base, deg, num_var):
    if deg % 2 != 0:
        print("No need to experiment polynomial with even degree")
        return 0
    r = m_mtx_base(deg = deg / 2, var_num = num_var)
    m_mtx = gen_mmoment_matrix(r.base_vec)
    base = mono_poly(deg=deg, var_num = num_var)
    A = A_mtx(m_mtx, base_type=base, base=base.base)
    c_num = condition_num(A)
    return c_num, c_num.evalf()

In [187]:
experiment(mono_poly, mono_poly, 4, 2)

TypeError: 'float' object cannot be interpreted as an integer